<a href="https://colab.research.google.com/github/juDEcorous/ML-Classification/blob/main/Exercise_Classification_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This task uses a breast cancer data set that can be found here. The target vector is the diagnosis as either malignant (M) or benign (B).

Your task is to use a

1. decision tree classifier,

2. bagging classifier, and a

3. random forest classifier

to obtain the highest accuracy possible on the test set.

You may want to refer to the previous chapter to review bagging and random forest as they were explained with regression. They work very similarly in a classification problem, but the final prediction is a majority vote of which class instead of an average of continuous values.

Note, that the following code can be used to import the classification tree models required:

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import r2_score

In [ ]:
path = '/content/drive/MyDrive/Datas/cancer.csv'
df = pd.read_csv(path, index_col=0)

In [ ]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 569 entries, 842302 to 92751
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se            

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
df.describe()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [ ]:
df['diagnosis'].value_counts(normalize = True)

B    0.627417
M    0.372583
Name: diagnosis, dtype: float64

In [ ]:
df['diagnosis'] = df['diagnosis'].replace({'B' : 0, 'M' : 1})
df['diagnosis'].value_counts(normalize = True)

0    0.627417
1    0.372583
Name: diagnosis, dtype: float64

# Slpit Validation

In [ ]:
target = 'diagnosis'
y = df[target]
X = df.drop(columns = target)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42) 

# Classification Evaluation



In [ ]:
def classification_evaluation(model, X_train, x_test, y_train, y_test, name='name'):

  train_score = model.score(X_train, y_train)
  test_score = model.score(X_test, y_test)

  print(f'{name} Training: {train_score}')
  print(f'{name} Testing: {test_score}')


# Decision Tree Classifier

In [ ]:
dec_tree = DecisionTreeClassifier(random_state = 42)
dec_tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [ ]:
classification_evaluation(dec_tree, X_train, X_test, y_train, y_test, name='Decision Tree Classification')

Decision Tree Classification Training: 1.0
Decision Tree Classification Testing: 0.951048951048951


# Bagging Tree Classifier

In [ ]:
bag_tree = BaggingClassifier(random_state = 42)
bag_tree.fit(X_train, y_train)

BaggingClassifier(random_state=42)

In [ ]:
classification_evaluation(bag_tree, X_train, X_test, y_train, y_test, name='Bagging Tree Classification')

Bagging Tree Classification Training: 0.9929577464788732
Bagging Tree Classification Testing: 0.951048951048951


Random Forest Classifier

In [ ]:
ran_for = RandomForestClassifier(random_state = 42)
ran_for.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
classification_evaluation(ran_for, X_train, X_test, y_train, y_test, name='Random Forest Classification')

Random Forest Classification Training: 1.0
Random Forest Classification Testing: 0.965034965034965
